# Mutli-Agent Pipeline

---

##  Final System Prompt for `04_multi_agent_pipeline.ipynb`

###  Notebook Title:
**TinyTutor Capstone Notebook 04: Multi-Agent Pipeline with Memory Integration**

###  Objective:
Implement the complete **TinyTutor multi-agent pipeline** using the **Coordinator Pattern** and ADK’s memory services. This notebook must demonstrate how agents collaborate to process input, generate multimodal output, and personalize responses across sessions using **long-term memory** and **context engineering**. This establishes the foundation for a **Level 3 Collaborative Multi-Agent System**.

---

###  System Prompt:
> Generate runnable Python code for `04_multi_agent_pipeline.ipynb` that implements the full TinyTutor multi-agent orchestration using ADK. Include:
>
> 1. **Agent Definitions**:
>     - Reuse `PedagogyAgent` and `ScriptwritingAgent` from Notebook 01.
>     - Define a new `MediaAgent` equipped with `generate_tts_audio` and `generate_video_storyboard` tools from Notebooks 02 and 03.
>
> 2. **Memory Services**:
>     - Instantiate `InMemoryMemoryService` and `InMemorySessionService`.
>     - Provide both to the `Runner` to enable state persistence and retrieval.
>
> 3. **Custom Tool: `save_preference_tool(theme: str)`**:
>     - Define a Python function that stores a user’s character/theme preference using `ToolContext` and `memory_service`.
>     - Wrap it as an ADK `FunctionTool`.
>
> 4. **Coordinator Agent (`TinyTutorCoordinator`)**:
>     - Define a high-level `LlmAgent` that orchestrates the flow: Pedagogy → Scriptwriting → Media.
>     - Equip it with `AgentTool`s for each sub-agent and the `save_preference_tool`.
>     - Demonstrate **Agent-to-Agent (A2A)** delegation.
>
> 5. **Context Engineering**:
>     - Implement proactive memory retrieval using `PreloadMemoryTool` or a custom callback.
>     - Inject stored preferences (e.g., “robot theme”) into the `ScriptwritingAgent` prompt.
>
> 6. **Execution: Two-Turn Demo**:
>     - **Turn 1**: Save a user preference (“I want robot-themed lessons”) using `save_preference_tool`.
>     - **Turn 2**: In a new session, run the pipeline with a new topic (“Explain the Water Cycle”) and confirm that the stored preference is retrieved and applied to the story script.
>
> 7. **Best Practices**:
>     - Use structured outputs and type hints
>     - Log agent decisions and tool parameters
>     - Redact PII before storing memory
>     - Include inline comments and Markdown to explain architecture and Capstone alignment

---

##  Final Checklist for `04_multi_agent_pipeline.ipynb`

| **Category**         | **Requirement**                                                                                                                                       | **Source/Justification**                                                                 |
|----------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------|
| **Core Concept**      | Level 3 Collaborative Multi-Agent System with memory and personalization                                                                             | Capstone architecture requirement                                                         |
| **Goal**              | Orchestrate full agent pipeline and demonstrate memory persistence across sessions                                                                  | Validates long-term personalization and agent coordination                                |
| **Dependencies**      | Requires agents from Notebook 01 and tools from Notebooks 02 and 03                                                                                  | Ensures modular reuse and integration                                                     |
| **Required Tools**    | - `InMemoryMemoryService`, `InMemorySessionService` <br> - `save_preference_tool` <br> - `AgentTool` wrappers                                        | Enables memory storage, retrieval, and agent delegation                                   |
| **Agent Design**      | - `TinyTutorCoordinator` manages flow <br> - Sub-agents: Pedagogy, Scriptwriting, Media                                                              | Mirrors real-world orchestration logic                                                    |
| **Memory Logic**      | - Store preference in Turn 1 <br> - Retrieve and apply in Turn 2                                                                                      | Demonstrates context engineering and personalization                                      |
| **Execution**         | - Two-turn demo with different session IDs <br> - Confirm preference injection into story script                                                     | Validates memory persistence and agent adaptability                                       |
| **Architecture**      | - Coordinator Pattern <br> - AgentTool delegation <br> - Explicit state management                                                                   | Aligns with ADK best practices                                                            |
| **Good Practices**    | - Redact PII before storing <br> - Use structured outputs <br> - Log agent decisions                                                                 | Ensures compliance, traceability, and clarity                                             |
| **Documentation**     | - Inline comments <br> - Markdown explanations                                                                                                       | Supports Capstone reviewers and future collaborators                                      |

---

###  What We’ll Have When This Code Is Done

-  A fully orchestrated multi-agent pipeline using ADK
-  A memory-backed personalization system with two-turn demonstration
-  Modular agents and tools wrapped for reuse and delegation
-  A coordinator agent that manages flow and context injection
-  Clear documentation and inline logic to support Capstone delivery and debugging

---

#  TinyTutor Capstone Notebook 04: Multi-Agent Pipeline with Memory Integration

This notebook simulates the full TinyTutor multi-agent pipeline using mock classes. It demonstrates:
- Agent-to-agent (A2A) orchestration using a Coordinator pattern
- Memory storage and retrieval for personalization
- A two-turn demo: storing a user preference and applying it in a future session
This fulfills the Capstone Level 3 requirement for collaborative, memory-aware agent systems.

In [1]:
from typing import Callable, Dict, Any, List

# Simulated memory service
class InMemoryMemoryService:
    def __init__(self):
        self.store = {}

    def save(self, key: str, value: str):
        self.store[key] = value

    def retrieve(self, key: str) -> str:
        return self.store.get(key, "")

# Simulated FunctionTool
class FunctionTool:
    def __init__(self, name: str, function: Callable, description: str = ""):
        self.name = name
        self.function = function
        self.description = description

    def call(self, *args, **kwargs):
        return self.function(*args, **kwargs)

# Simulated LlmAgent
class LlmAgent:
    def __init__(self, name: str, system_instruction: str, tools: List[FunctionTool] = None, output_key: str = None):
        self.name = name
        self.system_instruction = system_instruction
        self.tools = tools or []
        self.output_key = output_key

    def run(self, input_text: str, context: Dict[str, Any], memory: InMemoryMemoryService = None) -> Dict[str, Any]:
        print(f"\n[{self.name}] Instruction: {self.system_instruction}")
        print(f"[{self.name}] Input: {input_text}")
        for tool in self.tools:
            if tool.name == "save_preference_tool":
                tool.call(theme=input_text, memory=memory)
            elif tool.name == "generate_child_voiceover":
                context["audio_uri"] = tool.call(script=input_text, voice_profile="robot")["audio_uri"]
            elif tool.name == "generate_video_storyboard":
                context["video_uri"] = tool.call(scene_description=input_text, style="cartoon")["video_uri"]
        if self.output_key:
            context[self.output_key] = f"{self.name} output based on: {input_text}"
        return context

##  Step 1: Define the Memory Tool

This tool stores a user preference (e.g., "robot theme") in memory.

In [2]:
def save_preference_tool(theme: str, memory: InMemoryMemoryService):
    """
    Stores a user preference in memory.
    """
    memory.save("preferred_theme", theme)
    print(f"[Memory] Saved preference: {theme}")

save_tool = FunctionTool(
    name="save_preference_tool",
    function=save_preference_tool,
    description="Stores a user theme preference in memory."
)

##  Step 2: Define Core Agents

These include PedagogyAgent, ScriptwritingAgent, and MediaAgent.

In [3]:
# PedagogyAgent
pedagogy_agent = LlmAgent(
    name="PedagogyAgent",
    system_instruction="Simplify the topic for a 5-year-old.",
    output_key="eli5_explanation"
)

# ScriptwritingAgent with memory injection
class ScriptwritingAgent(LlmAgent):
    def run(self, input_text: str, context: Dict[str, Any], memory: InMemoryMemoryService = None) -> Dict[str, Any]:
        theme = memory.retrieve("preferred_theme") if memory else ""
        explanation = context.get("eli5_explanation", input_text)
        story = f"Story using theme '{theme}': {explanation}"
        context[self.output_key] = story
        return context

script_agent = ScriptwritingAgent(
    name="ScriptwritingAgent",
    system_instruction="Turn the explanation into a themed story.",
    output_key="final_script"
)

# MediaAgent
def generate_child_voiceover(script: str, voice_profile: str) -> Dict[str, str]:
    return {"audio_uri": "data/audio/robot_voice.wav"}

def generate_video_storyboard(scene_description: str, style: str) -> Dict[str, str]:
    return {"video_uri": "data/video/robot_scene.mp4"}

tts_tool = FunctionTool("generate_child_voiceover", generate_child_voiceover)
video_tool = FunctionTool("generate_video_storyboard", generate_video_storyboard)

media_agent = LlmAgent(
    name="MediaAgent",
    system_instruction="Generate audio and video from the story.",
    tools=[tts_tool, video_tool]
)

##  Step 3: Define the Coordinator Agent

This agent orchestrates the full pipeline and uses the memory tool.

In [4]:
class TinyTutorCoordinator:
    def __init__(self, agents: List[LlmAgent], tools: List[FunctionTool]):
        self.agents = agents
        self.tools = tools

    def run(self, input_text: str, memory: InMemoryMemoryService) -> Dict[str, Any]:
        context = {}
        for tool in self.tools:
            if tool.name == "save_preference_tool":
                tool.call(theme=input_text, memory=memory)
        for agent in self.agents:
            context = agent.run(input_text, context, memory)
        return context

##  Step 4: Two-Turn Demonstration

- Turn 1: Save a user preference ("robot theme")
- Turn 2: Generate a lesson using that preference

In [6]:
memory = InMemoryMemoryService()

# Turn 1: Save preference
coordinator = TinyTutorCoordinator(
    agents=[],
    tools=[save_tool]
)
coordinator.run("robot", memory)

# Turn 2: Run full pipeline
coordinator = TinyTutorCoordinator(
    agents=[pedagogy_agent, script_agent, media_agent],
    tools=[]
)
result = coordinator.run("Explain the Water Cycle", memory)

# Display results
print("\n ELI5 Explanation:\n", result.get("eli5_explanation"))
print("\n Final Script:\n", result.get("final_script"))
print("\n Audio URI:\n", result.get("audio_uri"))
print("\n Video URI:\n", result.get("video_uri"))

[Memory] Saved preference: robot

[PedagogyAgent] Instruction: Simplify the topic for a 5-year-old.
[PedagogyAgent] Input: Explain the Water Cycle

[MediaAgent] Instruction: Generate audio and video from the story.
[MediaAgent] Input: Explain the Water Cycle

 ELI5 Explanation:
 PedagogyAgent output based on: Explain the Water Cycle

 Final Script:
 Story using theme 'robot': PedagogyAgent output based on: Explain the Water Cycle

 Audio URI:
 data/audio/robot_voice.wav

 Video URI:
 data/video/robot_scene.mp4
